# Advanced File Parsing

Parse an sdf file with RegEx

In [ ]:
import pandas as pd
import numpy as np
import re

with open('SBE-b-CD-data.sdf') as f:
    sdf_string = f.read()
    
    # Info for molecules is given between $$$$
    pattern = re.compile('(?<=\$\$\$\$)((.|\n)*?)(?=\$\$\$\$)')
    matches = pattern.finditer(sdf_string)
    

In [ ]:
#print(f'{len(list(matches))} molecules found in file')

In [ ]:
molecule_info = []

for match in matches:
    
    # Strip to get rid of whitespace around the text.
    molecule_string = str(match.group().strip())
    
    # Get the molecule name
    molecule_name = molecule_string[:molecule_string.find('\n')]
    
    # Set up dictionary
    one_info = {}
    one_info['name'] = molecule_name
    
    
    # Get any key-value properties
    key_pattern = re.compile('\<(.*)\>\s[^\n]*')
    key_string = key_pattern.finditer(molecule_string)
    
    for k in key_string:
        search_string = k.group()
        
        key = re.search('(?<=\<)((.|\n)*?)(?=\>)', search_string).group()
        end_key = re.search('(?<=\<)((.|\n)*?)(?=\>)', search_string).span()[-1]
        value = search_string[end_key+1:].strip()

        try:
            one_info[key] = float(value)
        except ValueError:
            one_info[key] = value
    
    # Find the molecule coordinates.
    pattern = re.compile('([-,+]?[0-9]\.[0-9]{4}\s*){3}(\D)')
    coordinates = pattern.finditer(molecule_string)
    
    # Get coordinates and element name.
    molecule_coordinates = []
    molecule_elements = []
    for coordinate in coordinates:
        coord_info = coordinate.group().split()
        element = coord_info[-1]
        coordinates = coord_info[:-1]
        molecule_coordinates.append(coordinates)
        molecule_elements.append(element)
    
    one_info['coordinates'] = np.array(molecule_coordinates)
    one_info['elements'] = molecule_elements

    molecule_info.append(one_info)

In [ ]:
df = pd.DataFrame.from_dict(molecule_info)

In [ ]:
df